In [ ]:
import base64
import pandas as pd
from smolagents import tool, ToolCallingAgent, CodeAgent, PromptTemplates, OpenAIServerModel #TransformersModel, HfApiModel
from jaws.jaws_config import *
from jaws.jaws_utils import dbms_connection

In [2]:
driver = dbms_connection(DATABASE)

In [3]:
@tool
def fetch_data_for_advisor() -> pd.DataFrame:
    """
    Fetches the latest data from the database and returns it as a pandas DataFrame and JSON.

    Returns:
        tuple: A tuple containing the pandas DataFrame and JSON representation of the data.
    """
    query = """
    MATCH (traffic:TRAFFIC)
    RETURN DISTINCT
        traffic.IP_ADDRESS AS ip_address,
        traffic.PORT AS port,
        traffic.ORGANIZATION AS org,
        traffic.HOSTNAME AS hostname,
        traffic.LOCATION AS location,
        traffic.TOTAL_SIZE AS total_size,
        traffic.OUTLIER AS outlier
    """
    with driver.session(database=DATABASE) as session:
        result = session.run(query)
        data = []
        for record in result:
            data.append({
                'ip_address': record['ip_address'],
                'port': record['port'],
                'org': record['org'],
                'hostname': record['hostname'],
                'location': record['location'],
                'total_size': record['total_size'],
                'outlier': record['outlier']
            })
        return pd.DataFrame(data)

In [4]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
EVIDANCE = f"{FINDER_ENDPOINT}/pca_dbscan_outliers.png"
base64_image = encode_image(EVIDANCE)

In [5]:
expert_analysis = ToolCallingAgent(
    name="network_security_advisor",
    description="Reviews network traffic and provides recommendations for firewall configurations.",
    model=OpenAIServerModel(model_id=OPENAI_MODEL),
    #planning_interval=1,
    max_steps=4,
    tools=[fetch_data_for_advisor]
)

In [ ]:
analysis = expert_analysis.run(ADVISOR_PROMPT)